In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json

In [6]:
def perform_ner(text):
    prompt = f"""
    Perform Named Entity Recognition (NER) on the text below. Follow these rules STRICTLY:
    1. Identify only these entity types: PERSON, ORGANIZATION, LOCATION, DATE.
    2. Output MUST be valid JSON format matching this exact schema:
    ```json
    [
        {{"entity": "entity_name", "type": "ENTITY_TYPE"}}
    ]
    3. Do NOT include any explanations, notes, or non-JSON text.
    4. Do NOT return the given text to perform NER on, give only the JSON output.

    5.If no entities exist, return empty list [].

    6.Text: "{text}"

    7.DO NOT PRINT ANYTHING OTHER THAN OUTPUT JSON:"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "deepseek-r1:14b",
            # "model": "llama2:13b",
            "prompt": prompt,
            "stream": False
        }
    )
    result = response.json()
    print("Ollama API response:", result)
    # Ollama returns {'response': '...'}; extract and parse JSON from output
    output = result['response']
    try:
        # Try to extract the JSON array from the output
        entities = json.loads(output[output.find('['):output.rfind(']')+1])
    except Exception as e:
        entities = []
    return entities

In [7]:
text = """
Tim Cook (born November 1, 1960, Mobile, Alabama, U.S.) is an American technology executive who has been the chief executive officer (CEO) of the technology company Apple Inc. since 2011.
Cook grew up in Robertsdale, Alabama. He graduated from Auburn University in Alabama with a bachelor’s degree in industrial engineering in 1982, and in 1988 he received a master’s in business administration from Duke University in Durham, North Carolina. He worked for the computer manufacturer International Business Machines Corporation (IBM) from 1982 to 1994, with his final position being director of North American fulfillment. He was subsequently chief operating office of the reseller division at the computer retailer Intelligent Electronics, Inc. (1994–97), and vice president of corporate materials at the computer manufacturer Compaq Computer Corporation (1997).
"""

# print(perform_ner(text))
print("Model reply:", perform_ner(text))


Ollama API response: {'model': 'deepseek-r1:14b', 'created_at': '2025-05-13T06:01:51.663368296Z', 'response': '<think>\nOkay, I need to perform Named Entity Recognition on the given text following specific rules. Let me go through it step by step.\n\nFirst, the user has provided a detailed text about Tim Cook and his career. My task is to identify entities of types PERSON, ORGANIZATION, LOCATION, or DATE only.\n\nI\'ll start reading the text carefully:\n\n- "Tim Cook" is clearly a PERSON.\n- "(born November 1, 1960, Mobile, Alabama, U.S.)": Here, November 1, 1960 is a DATE. Mobile and Alabama are both LOCATIONS (cities and states).\n- "Apple Inc." is an ORGANIZATION.\n- He grew up in Robertsdale, which is another LOCATION (city) in Alabama.\n- "Auburn University" is an ORGANIZATION (institution), located in Alabama.\n- "Duke University" is an ORGANIZATION in Durham, North Carolina. So, Durham and North Carolina are LOCATIONS.\n- IBM is an ORGANIZATION.\n- "Intelligent Electronics, Inc.